<a href="https://colab.research.google.com/github/amifunny/Deep-Learning-Notebook/blob/master/Music_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.1.0

In [0]:
from music21 import *
import numpy as np
import tensorflow as tf
import os

In [0]:
# download data form her == >'https://drive.google.com/file/d/1qnQVK17DNVkU19MgVA4Vg88zRDvwCRXw/view?usp=sharing'
# and upload 'schubert.zip' to this notebook

In [0]:
!unzip schubert.zip -d midi

In [0]:
#defining function to read MIDI files
def read_midi(file):
    
    print("Loading Music File:",file)
    
    notes=[]
    notes_to_parse = None
    
    #parsing a midi file
    midi = converter.parse(file)
  
    #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)

    #Looping over all the instruments
    for part in s2.parts:
    
        #select elements of only piano
        if 'Piano' in str(part): 
        
            notes_to_parse = part.recurse() 
      
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                
                #note
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                
                #chord
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

In [0]:
base_dir = '/content/midi/'
all_midi_files = os.listdir( base_dir )
song_list = []

for each_file in all_midi_files:
    
    song_list.append( read_midi( base_dir + each_file ) )

print( len(all_midi_files) )
print( len(song_list) )

In [0]:
print( song_list[5].shape )

In [0]:
#get the vocab
vocab_set = set()

for each_song in song_list:
    
    vocab_set.update( each_song )

vocab_list = list(vocab_set)
print(vocab_list)
print( len(vocab_list) )    

In [0]:
# get a dictionery
int_to_note = {}
note_to_int = {}

for i in range( len(vocab_list) ):

    int_to_note[i] = vocab_list[i]
    note_to_int[ vocab_list[i] ] = i

print(int_to_note)
print(note_to_int)
print( len(note_to_int) )

# #########################
vocab_size = len(note_to_int)


In [0]:
# convert note seq to int seq
int_x_seq=[]
int_y_seq=[]

# what we want model to learn is predict
# next node after seeing some "timestep nodes"
# ie after seeing 32 notes predict 33rd
timesteps = 32

for each_song in song_list:

    num_of_chunks = each_song.shape[-1]-timesteps
    for i in range(num_of_chunks):
        
        temp_x = [ note_to_int[w] for w in each_song[i:i+timesteps] ]
        temp_y = note_to_int[ each_song[i+timesteps] ]
        
        int_x_seq.append( temp_x )
        int_y_seq.append( temp_y )

print(len(int_x_seq))
print(len(int_y_seq))

In [0]:
ind = 6
print( int_x_seq[ind] )
print( int_y_seq[ind] )
print( len(int_x_seq[ind]) )
print( type(int_x_seq[ind]) )


In [0]:
model = 0
def get_model():

    model =  tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size,16),
          tf.keras.layers.LSTM(32,
                        return_sequences=True),
          tf.keras.layers.LSTM(32,
                        return_sequences=True),
          tf.keras.layers.LSTM(32),
          tf.keras.layers.Dense(64,activation='relu'),
          tf.keras.layers.Dense(vocab_size,activation='softmax') 
      ])

    return model

In [0]:
model = get_model()
model.summary()

In [0]:
shuffle_x = []
shuffle_y = []

perm_index = np.random.permutation( len(int_x_seq) )

for i in perm_index:
  shuffle_x.append( int_x_seq[i] )
  shuffle_y.append( int_y_seq[i] )


In [0]:
num_of_val = 1000

x_tf = tf.convert_to_tensor(shuffle_x[:-num_of_val])
y_tf = tf.convert_to_tensor(shuffle_y[:-num_of_val])

print(x_tf.shape)

val_x = shuffle_x[-num_of_val:-1]
print(len(val_x))


In [0]:
print( x_tf[5] )
print( y_tf[5] )

print( x_tf[6] )
print( y_tf[6] )

In [0]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy( )
optimizer = tf.keras.optimizers.Adam(0.0005)
mean = tf.keras.metrics.Mean()

batch_size = 32
num_of_batches = int( x_tf.shape[0]/batch_size )
epochs = 10

with tf.device('/device:GPU:0'):
  
  for e in range(epochs):

    ctr = 0
    mean.reset_states()

    for i in range( num_of_batches ):  

      x_batch = x_tf[i*batch_size:(i+1)*batch_size]
      y_batch = tf.expand_dims( y_tf[i*batch_size:(i+1)*batch_size] , -1 )

      with tf.GradientTape() as tape:
        
        pred = model( x_batch )
        loss = loss_func( y_batch , pred )

      mean.update_state(loss)

      grad = tape.gradient(loss,model.trainable_variables)
      optimizer.apply_gradients(zip(grad,model.trainable_variables))

      if ctr%500==0:
          print("loss is  ===>  {}".format(mean.result()))

          print( tf.squeeze(y_batch) )
          print( tf.squeeze( tf.argmax(pred,-1) ) )


      ctr = ctr + 1

    print("Epoch {} === > {}".format(e,mean.result()))



In [0]:
model.trainable_variables

In [0]:
model.save('music_model.h5')

In [0]:
def gen_music(start_salt):

    gen_steps = 200
    
    music_step = len(start_salt)
    # print( music_step )

    pred_list = start_salt

    # print(pred_list)
    temperature=0.1

    for i in range( gen_steps ):

      init_music = np.array(pred_list[:i+music_step] )
      init_tf = tf.expand_dims( tf.convert_to_tensor( init_music ) , 0 )

      # print(init_tf)
      pred_one = model(init_tf)
      pred_hot = tf.squeeze( tf.argmax(pred_one,-1) )

      predictions = pred_one / temperature
      pred_val = tf.random.categorical(predictions, num_samples=1)


      pred_val = (tf.squeeze(pred_val)).numpy()
      pred_list.append( pred_val  )

    return pred_list

In [0]:
temp_init = val_x[-5][:-1]
my_music = gen_music(temp_init)

In [0]:
note_music = []
for each_ele in my_music:

    note_music.append(int_to_note[each_ele])

print(note_music)

In [0]:
# this is midi related function that i took from internet it simply turn vocab into music
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [0]:
convert_to_midi(note_music)
# download the music.mid file from files explorer on left in colab and listen to your own sweet creation